# Capstone - Project Definition
## Author - Salinee Kingbaisomboon
### UW NetID: 1950831

For this capstone project, we will use the **SECOM** manufacturing Data Set from the UCI Machine Learning Repository. The set is originally for semiconductor manufacturing, but in our case, we will assume that it is for the diaper manufacturing process.

The dataset consists of two files:
1. A dataset file SECOM containing 1567 examples, each with 591 features, presented in a 1567 x 591 matrix
2. A labels file listing the classifications and date time stamp for each example

## Instructions
Assume the 591 attributes represent different sensors readings across the manufacturing process.

1. Refine your data understanding and prepare a DFD of solving the manufacturing quality problem
2. Read and merge data
3. Clean and prepare data
4. Visually explore data
5. Handle class imbalance problem
6. Apply feature selection techniques to reduce dimensionality of data
7. Describe your initial findings in a summary section
8. Solicit specific feedback on your code

## Data Understandins and Data Flow Diagram (DFD)
The **SECOM manufacturing Data Set** is originally for semiconductor manufacturing, but in our case, we will assume that it is for the diaper manufacturing process.

Based on https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.names, this data set has <font color=red>**missing values**</font>. Therefore, we need to clean data as well.

Also, these data set doesn't come with header definition. Therefore, we have to create pseudo features names for all attribues in order to be able to perform data analysis.

Moreover, not all of these signals are equally valuable  in a specific monitoring system. The measured signals contain a combination of useful information, irrelevant information as well as noise. We have to perform some <font color=blue>**feature engineering**</font> to reduce dimensionality of data.

![](https://i.ibb.co/PMZznZM/DFD.png)

**Link: https://i.ibb.co/PMZznZM/DFD.png**

In [66]:
# Load necessary libraries
import pandas as pd
import numpy as np

from collections import Counter

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE # Recursive Feature Elimination (for backward model selection)

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore") # To suppress warning

%matplotlib inline

pd.options.display.max_rows = None
pd.options.display.max_columns = None

# Declare Functions used in this milestone

In [36]:
# Function to replace missing numeric values
def replace_missing_value(x, col):
    # We determine the locations of NaN
    HasNan = np.isnan(x.loc[:, col]) 
    # If there are NaN
    if sum(HasNan) > 0:
        # Calculate the Median for current column without Nan
        Median = np.nanmedian(x.loc[:, col])
        # Replace the missing value with Median
        x.loc[HasNan, col] = Median
    return x.loc[:, col]

In [56]:
# Load data and label
dataFilename = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data'
data_df = pd.read_csv(dataFilename, header=None, delim_whitespace=True)

labelFilename = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data'
label_df = pd.read_csv(labelFilename, header=None, delim_whitespace=True)

In [57]:
# Set pseudo names for input data
for col in data_df.columns:
    data_df.rename(columns = {col:'feature_'+ str(col)}, inplace = True)
    
# Set pseudo name for label data
label_df.rename(columns = {0:'HouseLineTestingResult'}, inplace = True)
label_df.rename(columns = {1:'TimeStamp'}, inplace = True)

# Since we didn't do the time series analysis, we don't need the timestamp column
# Drop the unused columns
label_df.drop(['TimeStamp'],axis=1, inplace=True)

In [58]:
# Clean input data
for col in data_df.columns:
    # Replace missing values with Median for current column without Nan
    data_df.loc[:, col] = replace_missing_value(data_df, col)

In [59]:
# Merge input and label data into one
df = pd.concat([data_df,label_df],axis=1)

## Explore Data

In [60]:
# View first 5 rows
df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_581,feature_582,feature_583,feature_584,feature_585,feature_586,feature_587,feature_588,feature_589,HouseLineTestingResult
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,72.2889,0.5005,0.0118,0.0035,2.3630,0.0205,0.0148,0.0046,71.9005,-1
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,...,72.2889,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1


In [42]:
print(df.shape) # Print DataFrame's size
print(df.dtypes) # Print DataFrame's data types

(1567, 592)
feature_0                        float64
feature_1                        float64
feature_2                        float64
feature_3                        float64
feature_4                        float64
feature_5                        float64
feature_6                        float64
feature_7                        float64
feature_8                        float64
feature_9                        float64
feature_10                       float64
feature_11                       float64
feature_12                       float64
feature_13                       float64
feature_14                       float64
feature_15                       float64
feature_16                       float64
feature_17                       float64
feature_18                       float64
feature_19                       float64
feature_20                       float64
feature_21                       float64
feature_22                       float64
feature_23                       float64
feat

In [43]:
# View data frame columns
df.columns

Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       ...
       'feature_582', 'feature_583', 'feature_584', 'feature_585',
       'feature_586', 'feature_587', 'feature_588', 'feature_589',
       'HouseLineTestingResult', 'TimeStamp'],
      dtype='object', length=592)

In [67]:
# View Statistic Summary
df.describe()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200,feature_201,feature_202,feature_203,feature_204,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,feature_219,feature_220,feature_221,feature_222,feature_223,feature_224,feature_225,feature_226,feature_227,feature_228,feature_229,feature_230,feature_231,feature_232,feature_233,feature_234,feature_235,feature_236,feature_237,feature_238,feature_239,feature_240,feature_241,feature_242,feature_243,feature_244,feature_245,feature_246,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,feature_257,feature_258,feature_259,feature_260,feature_261,feature_262,feature_263,feature_264,feature_265,feature_266,feature_267,feature_268,feature_269,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276,feature_277,feature_278,feature_279,feature_280,feature_281,feature_282,feature_283,feature_284,feature_285,feature_286,feature_287,feature_288,feature_289,feature_290,feature_291,feature_292,feature_293,feature_294,feature_295,feature_296,feature_297,feature_298,feature_299,feature_300,feature_301,feature_302,feature_303,feature_304,feature_305,feature_306,feature_307,feature_308,feature_309,feature_310,feature_311,feature_312,feature_313,feature_314,feature_315,feature_316,feature_317,feature_318,feature_319,feature_320,feature_321,feature_322,feature_323,feature_324,feature_325,feature_326,feature_327,feature_328,feature_329,feature_330,feature_331,feature_332,feature_333,feature_334,feature_335,feature_336,feature_337,feature_338,feature_339,feature_340,feature_341,featu

## Handle Class Imbalance

In [61]:
# Split test(90%) and train (10%) data
targetOutcome = pd.DataFrame(df,columns=['HouseLineTestingResult'])
allInputs = pd.DataFrame(df,columns=df.columns.difference(['HouseLineTestingResult']))
X, XX, Y, YY = train_test_split(allInputs, targetOutcome, test_size = 0.1, random_state = 0)

In [62]:
# Modify data by handling class imbalance using SMOTE method
print('Original training targets shape {}'.format(Counter(targetOutcome.values.ravel())))
sm = SMOTE(random_state=0)
allInputs_resample, targetOutcome_resample = sm.fit_sample(allInputs, targetOutcome.values.ravel())
print('Resampled dataset shape {}'.format(Counter(targetOutcome_resample)))

Original training targets shape Counter({-1: 1463, 1: 104})
Resampled dataset shape Counter({-1: 1463, 1: 1463})


## Feature Selection
I decide to use <font color=blue>**Backward Stepwise Model Selection**</font> which mean we start the model with all features included and we removes one feature on each iteration. The variables that can minimally increase the **Residual Sum of Squares (RSS)** on the data is chosen as the feature to be removed from the model.

In [65]:
# Create linear regression model as the estimator
estimator = LinearRegression()
# From the Filter method, we select 10 features -- this number is arbitary (need to find the optimal number with further analysis)
selector = RFE(estimator, 10, step=1)
# Learn from this dataset
selector = selector.fit(allInputs, targetOutcome)
# The mask of selected features (which variables are selected)
print(selector.support_)
# Selected features are ranked 1. The variable with the highest rank is the one that is removed first
print(selector.ranking_)
# Print the name of selected features
f = selector.get_support(True) # the most important features
print('\033[1m\033[4mSeledted features from Backward Stepwise Model Selection are: \033[0m')
for f_index in f:
    print(df.columns[f_index])

[False False False False  True False False False False False False False
 False False False False False False  True False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False  True False
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

***
**Summary:** Initial finding are:

- Based on **Statistic Summary**, we found that this data set has a wide range of values amoung each features. Therefore, we need to perform the **Normalization** for further analysis.
- Some features act like the constant and are not provide any benefit to the analysis at all such as 
    - **feature_5** which has constant value = 100
    - **feature_13** which has constant value = 0
    - **feature_42** which has constant value = 70
    - **feature_49** which has constant value = 1
- Based on an feature selection method using **Backward Stepwise Model Selection**, the model has select first ten important features:
    - feature_4
    - feature_18
    - feature_82
    - feature_84
    - feature_167
    - feature_232
    - feature_234
    - feature_288
    - feature_320
    - feature_585
***

***
**Solicit Feedback:**  

- I decide to drop the **TimeStamp** features from the data set since I stated that we didn't perform the **Times Series** analysis. My question is "Is it possible to perform the Time Series Analysis" to measure the manufacturing quality (Quality Control)? Is there any real scenario that there is a trend in the quality control of the production line? I feel like time of the year should not be related when the product will pass or failed.
- For **feature selection** using **Backward Stepwise Model**, I did **select 10 features** by arbitary. I did stated that I need to try to find the optimal number of this n features. Is the **Accuracy Measurement (AUC, Recall, Precision)** the right way (select the n features from the model that yeild the best accuracy)?
***